<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h1 style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">M3.3</span> – تتبّع التغييرات في الشيفرة البحثية</h1><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">جزء من: <a href="https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources" dir="ltr">Open Science for Water Resources</a></p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في القسم السابق عالجنا بيانات الهطول العالمية من <span dir="ltr" style="unicode-bidi: isolate;">IMERG-Final</span> إلى سلسلة زمنية لمنطقة الدراسة. في هذا الدرس سنجعل الشيفرة قابلة لإعادة الاستخدام، وسنتعلّم كيف نتتبّع تغييرات الشيفرة بطريقة قابلة للتدقيق باستخدام <span dir="ltr" style="unicode-bidi: isolate;">Git</span>.</p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تحويل الشيفرة إلى دوال ذات مُدخلات ومخرجات واضحة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تهيئة مستودع <span dir="ltr" style="unicode-bidi: isolate;">Git</span> وتسجيل التغييرات كـ <span dir="ltr" style="unicode-bidi: isolate;">commits</span>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">اعتماد رسائل تثبيت واضحة تساعد على فهم لماذا تغيّرت الشيفرة.</li>
</ul>
</div>

In [ ]:
import calendar
import datetime
import glob
import earthaccess
import numpy as np
import h5py
import xarray as xr
import geopandas
from matplotlib import pyplot
from pyproj import CRS

auth = earthaccess.login()

basin = geopandas.read_file('data/shp/YellowstoneRiver_drainage_WSG84.shp')

# NOTE: We don't want to re-download the data right now
# results = earthaccess.search_data(
#     short_name = 'GPM_3IMERGM',
#     temporal = ('2014-01-01', '2023-12-31'))
# earthaccess.download(results, 'data/IMERG-Final_monthly')

file_list = glob.glob('data/IMERG-Final_monthly/*.HDF5')
file_list.sort()

datasets = []
for i, filename in enumerate(file_list):
    # Only need to do this once, for the first file
    if i == 0:
        with h5py.File(filename, 'r') as hdf:
            longitude = hdf['Grid/lon'][:]
            latitude = hdf['Grid/lat'][:]

    # Get the date of this image
    date = datetime.datetime.strptime(filename.split('.')[4][0:8], '%Y%m%d')
    ds0 = xr.open_dataset(
        filename, group = 'Grid', decode_times = False).get(['precipitation'])
    # Define the missing coordinates
    ds0 = ds0.assign_coords({
        'time': [date], 'x': longitude, 'y': latitude
    })
    
    # Define the coordinate reference system (CRS) and the spatial coordinates
    ds0 = ds0.rio.write_crs(CRS.from_epsg(4326))
    ds0 = ds0.rio.set_spatial_dims('lon', 'lat')

    # Clip the IMERG-Final precipitation data to our basin's boundary
    ds_clip = ds0.rio.clip(basin.geometry.values)
    
    # Save the clipped dataset to be merged with the others
    datasets.append(ds_clip)

# Merge the datasets together along the "time" axis (i.e., build a time series)
ds = xr.concat(datasets, dim = 'time')

# Converting from [mm hour-1] to [mm month-1]
days_in_month = np.array(calendar.mdays)[ds.coords['time.month'].values]
ds['precip_monthly'] = ds.precipitation * 24 * days_in_month.reshape((days_in_month.size, 1, 1))

# Compute basin-wide monthly precipitation
precip_series = ds.precip_monthly.mean(['lon','lat']).values

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تكييف الشيفرة البحثية لإعادة الاستخدام</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سنبدأ بتحويل الخطوات المتكررة إلى دوال، وتقليل الاعتماد على «ثوابت» داخل الشيفرة، حتى يصبح من السهل إعادة تشغيل التحليل على منطقة أو فترة زمنية مختلفة.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🏁 تحدٍّ: إعادة كتابة الشيفرة كدالة</h4><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أعد كتابة الشيفرة التي تُنشئ سلسلة الهطول الزمنية كدالة تأخذ المُدخلات اللازمة صراحةً (مثل مسار الملف، الحدود المكانية، أو اسم المتغيّر) وتُرجع مخرجات محددة (مثل <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> أو <span dir="ltr" style="unicode-bidi: isolate;">pandas.Series</span>).</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الدوال عادةً تحوّل مُدخلات (Arguments) إلى مخرجات (Return value). من الأفضل أن تكون الدالة قصيرة، واضحة، وقابلة للاختبار، وأن تقلّل الآثار الجانبية مثل الكتابة إلى القرص إلا عند الحاجة.</p>
</div>

In [ ]:
def hdf5_to_xarray_dataset(filename, longitude = None, latitude = None):
    '''
    Reads an HDF5 file representing daily data and returns an 
    xarray.Dataset with the date, latitude, and longitude coordinates
    properly defined.

    Parameters
    ----------
    filename : str
        The file path to the HDF5 file
    longitude : numpy.ndarray
        The longitude coordinates, as a 1D NumPy array
    latitude : numpy.ndarray
        The latitude coordinates, as a 1D NumPy array

    Returns
    -------
    xarray.Dataset
    '''
    if longitude is None or latitude is None:
        with h5py.File(filename, 'r') as hdf:
            longitude = hdf['Grid/lon'][:]
            latitude = hdf['Grid/lat'][:]

    # Get the date of this image
    date = datetime.datetime.strptime(filename.split('.')[4][0:8], '%Y%m%d')
    ds0 = xr.open_dataset(
        filename, group = 'Grid', decode_times = False).get(['precipitation'])
    # Define the missing coordinates
    ds0 = ds0.assign_coords({
        'time': [date], 'x': longitude, 'y': latitude
    })
    
    # Define the coordinate reference system (CRS) and the spatial coordinates
    ds0 = ds0.rio.write_crs(CRS.from_epsg(4326))
    ds0 = ds0.rio.set_spatial_dims('lon', 'lat')
    return ds0

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h4 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">🎯 أفضل الممارسات</h4><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مرّر الإعدادات كوسائط بدل الاعتماد على متغيّرات عالمية.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">استخدم أسماء متغيّرات واضحة وتجنّب الغموض.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قلّل تكرار الشيفرة عبر إعادة استخدام الدوال.</li>
</ul>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr/><h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تتبع الإصدارات باستخدام <span dir="ltr" style="unicode-bidi: isolate;">Git</span></h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الآن سنستخدم نظام تتبع الإصدارات لتسجيل التغييرات على الشيفرة. الهدف هو معرفة أي نسخة من الشيفرة أنتجت أي نتيجة، والقدرة على الرجوع لنسخ سابقة عند الحاجة.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">هل تتذكر هذه الأسطر المهمة؟ عندما نعيد تشغيل التحليل لاحقًا، نريد معرفة أي نسخة من الشيفرة أنتجت أي نتيجة. لذلك نحتاج تتبع الإصدارات.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مقدمة إلى <span dir="ltr" style="unicode-bidi: isolate;">Git</span></h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">Git</span> هو نظام لتتبع التغييرات في الملفات النصية. يسمح بإنشاء «لقطات» (Commits) تمثل نقاطًا واضحة في تاريخ المشروع، مع رسالة تصف ما تغيّر ولماذا.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">اختيار مكان مناسب لمستودع Git</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">اختر مجلد مشروع واضح يحتوي الشيفرة وملفات التوثيق. حاول فصل البيانات الخام والمخرجات عن الشيفرة، وتجنّب إدراج ملفات ضخمة غير ضرورية داخل المستودع.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>إذا كنت قد بدأت العمل داخل مجلد المشروع بالفعل</strong>، فتأكد من أن بنية المجلدات منطقية قبل تهيئة <span dir="ltr" style="unicode-bidi: isolate;">Git</span>، لأن <span dir="ltr" style="unicode-bidi: isolate;">Git</span> سيتتبع ما يوجد داخل هذا المجلد.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تهيئة مستودع Git</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لتهيئة مستودع جديد داخل مجلد المشروع نستخدم الأمر <span dir="ltr" style="unicode-bidi: isolate;">git init</span>.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إضافة الملفات إلى المستودع</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أضف الملفات التي تريد تتبعها باستخدام <span dir="ltr" style="unicode-bidi: isolate;">git add</span>، ثم أنشئ تثبيتًا باستخدام <span dir="ltr" style="unicode-bidi: isolate;">git commit</span> مع رسالة واضحة.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إنهاء التغييرات</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كرر دورة: (تعديل → <span dir="ltr" style="unicode-bidi: isolate;">git add</span> → <span dir="ltr" style="unicode-bidi: isolate;">git commit</span>) كلما وصلت إلى نقطة منطقية. اكتب رسائل تثبيت تصف «ما الذي تغيّر» و«لماذا».</p>
</div>